In [17]:
import pandas as pd
from neo4j import GraphDatabase
from neo4j.exceptions import AuthError
import json
from tqdm import tqdm
from typing import Optional, Dict
import re

In [18]:
def prepare_llm_data(results_df: pd.DataFrame, json_column_name: str) -> Optional[pd.DataFrame]:
    """Prepares LLM results by extracting entities from the specified JSON column."""
    
    comparison_list = []
    for index, row in results_df.iterrows():
        try:
            json_string = row[json_column_name]
            if pd.isna(json_string): continue
            llm_json_data = json.loads(json_string)
            
            extracted_entities = {
                "UniqueAccident": None, "AccidentType": None, "TrackSection": None,
                "Date": None, "Time": None, "Country": None, "RegulatoryBody": None,
                "ContributingFactor": [], "SystemicFactor": []
            }
            for node in llm_json_data.get("nodes", []):
                node_type, node_id = node.get("type"), node.get("id")
                if node_type in extracted_entities:
                    if isinstance(extracted_entities[node_type], list):
                        if node_id is not None: extracted_entities[node_type].append(str(node_id))
                    elif extracted_entities[node_type] is None and node_id is not None:
                        extracted_entities[node_type] = str(node_id)
            
            for factor_type in ["ContributingFactor", "SystemicFactor"]:
                if extracted_entities[factor_type]:
                    extracted_entities[factor_type] = ", ".join(sorted(list(set(extracted_entities[factor_type]))))
                else:
                    extracted_entities[factor_type] = None

            erail_id_match = re.search(r'([A-Z]{2}-\d+)', str(row["pdf_name"]))
            erail_occurrence_id = erail_id_match.group(1).upper() if erail_id_match else None
            
            entry = {
                "pdf_name": row["pdf_name"], "model_type": row.get("model_type"),
                "iteration_number": row.get("iteration_number"),
                "ERAIL Occurrence": erail_occurrence_id
            }
            
            for entity_type, value in extracted_entities.items():
                entry[f"LLM_{entity_type}"] = value
            comparison_list.append(entry)
        except (json.JSONDecodeError, TypeError):
            continue
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            continue
        
    if not comparison_list: return pd.DataFrame()
    return pd.DataFrame(comparison_list)

In [19]:
df = pd.read_csv('../thesis_modularized/data/pdf_processing_results.csv')
# df = prepare_llm_data(df, "refined_output")

In [20]:
df

,pdf_name,model_type,iteration_number,extraction_output,refined_output
0,PL-1378.pdf,gemini-2.5-pro,1,"{\n ""nodes"": [\n {\n ""id"": ""Sprowa-St...","{\n ""nodes"": [\n {\n ""id"": ""Sprowa-St..."
1,PL-5488.pdf,gemini-2.5-pro,1,"{\n ""nodes"": [\n {\n ""id"": ""Accident ...","{\n ""nodes"": [\n {\n ""id"": ""Accident ..."
2,PL-1378.pdf,gemini-2.5-pro,2,"{\n ""nodes"": [\n {\n ""id"": ""Sprowa-St...","{\n ""nodes"": [\n {\n ""id"": ""Sprowa-St..."
3,BE-10535.pdf,gemini-2.5-pro,1,"{\n ""nodes"": [\n {\n ""id"": ""Collision...","{\n ""nodes"": [\n {\n ""id"": ""Collision..."
4,PL-10513.pdf,gemini-2.5-pro,1,"{\n ""nodes"": [\n {\n ""id"": ""Rogo\u017...","{\n ""nodes"": [\n {\n ""id"": ""Rogo\u017..."
...,...,...,...,...,...
216,BE-1406.pdf,gemini-2.5-pro,1,"{\n ""nodes"": [\n {\n ""id"": ""Remersdaa...","{\n ""nodes"": [\n {\n ""id"": ""Remersdaa..."
217,AT-0134.pdf,gemini-2.5-pro,1,"{\n ""nodes"": [\n {\n ""id"": ""Zusammenp...","{\n ""nodes"": [\n {\n ""id"": ""Zusammenp..."
218,AT-0143.pdf,gemini-2.5-pro,1,"{\n ""nodes"": [\n {\n ""id"": ""Collision...","{\n ""nodes"": [\n {\n ""id"": ""Collision..."
219,AT-0081.pdf,gemini-2.5-pro,1,"{\n ""nodes"": [\n {\n ""id"": ""Collision...","{\n ""nodes"": [\n {\n ""id"": ""Collision..."


In [21]:
# Neo4j Connection Setup
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"
NEO4J_DATABASE = "neo4j"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

try:
    # Test the connection
    with driver.session() as session:
        session.run("RETURN 1")
    print("Connected to Neo4j successfully.")
except AuthError as e:
    print("Authentication failed. Check your credentials:", str(e))

Connected to Neo4j successfully.


In [22]:
def push_to_neo4j(df, model, iteration, use_refined=True):

    df = df[(df["model_type"] == model) & (df["iteration_number"] == iteration)]
    
    column = "refined_output" if use_refined else "extraction_output"
    
    for _, row in df.iterrows():
        json_str = row.get(column)
        if not json_str:
            continue

        try:
            data = json.loads(json_str)
        except json.JSONDecodeError:
            continue

        with driver.session(database=NEO4J_DATABASE) as session:
            for node in data.get("nodes", []):
                session.run(
                    f"MERGE (n:`{node.get('type', 'Unknown')}` {{id: $id}}) "
                    "ON CREATE SET n.name = $id",
                    id=node.get("id")
                )

            for rel in data.get("rels", []):
                session.run(
                    f"MATCH (a {{id: $source}}) MATCH (b {{id: $target}}) "
                    f"MERGE (a)-[r:`{rel.get('type', 'RELATED_TO')}`]->(b)",
                    source=rel.get("source"),
                    target=rel.get("target")
                )

In [23]:
push_to_neo4j(df, model="gemini-2.5-pro", iteration=1, use_refined=True)

In [16]:
# Clear database
def clear_neo4j_database():
    """Delete all nodes and relationships in the Neo4j database."""
    with driver.session(database=NEO4J_DATABASE) as session:
        session.run("MATCH (n) DETACH DELETE n")
    print("Neo4j database cleared successfully.")

# Run the function to clear the database
clear_neo4j_database()

Neo4j database cleared successfully.


In [ ]:
# Close Neo4j connection
driver.close()

In [58]:
def run_query(cypher_query):
    with driver.session() as session:
        result = session.run(cypher_query)
        return pd.DataFrame([record.data() for record in result])

In [67]:
drop_query = "CALL gds.graph.drop('thesis', false)"
run_query(drop_query)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('schema' returned by 'gds.graph.drop' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL gds.graph.drop('thesis', false)"


,graphName,database,databaseLocation,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema,schemaWithOrientation
0,thesis,neo4j,local,,-1,105,314,{'relationshipProjection': {'systemic_factor':...,0.028755,2025-06-14T12:14:52.391495100+02:00,2025-06-14T12:14:52.391495100+02:00,"{'graphProperties': {}, 'nodes': {'Contributin...","{'graphProperties': {}, 'nodes': {'Contributin..."


In [70]:
create_projection_query = """
CALL gds.graph.project(
  'thesis',
  ['UniqueAccident', 'AccidentType', 'ContributingFactor', 'SystemicFactor', 'Country', 'Date', 'Time', 'TrackSection', 'RegulatoryBody'],
  {
    contributing_factor: { type: 'contributing_factor', orientation: 'NATURAL' },
    systemic_factor:     { type: 'systemic_factor',     orientation: 'NATURAL' },
    occurred_in:         { type: 'occurred_in',         orientation: 'NATURAL' },
    is_type:             { type: 'is_type',             orientation: 'NATURAL' },
    is_track_section:    { type: 'is_track_section',    orientation: 'NATURAL' },
    investigated_by:     { type: 'investigated_by',     orientation: 'NATURAL' },
    has_date:            { type: 'has_date',            orientation: 'NATURAL' },
    has_time:            { type: 'has_time',            orientation: 'NATURAL' }
  }
)
"""

projection_result = run_query(create_projection_query)
projection_result

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,{'ContributingFactor': {'label': 'Contributing...,"{'systemic_factor': {'aggregation': 'DEFAULT',...",thesis,105,157,69


In [81]:
query ="""
MATCH (start_node:ContributingFactor {id: "Awareness"})-[r]-(connected)
RETURN 
  labels(connected)[0] AS connected_type,
  connected.id AS connected_id,
  type(r) AS rel_type
"""

run_query(query)

,connected_type,connected_id,rel_type
0,UniqueAccident,Chałupki - Krzyżanowice Level Crossing Acciden...,contributing_factor
1,UniqueAccident,Komorów – Podkowa Leśna Główna Accident on 19/...,contributing_factor
2,UniqueAccident,Skierniewice Accident 24/08/2023,contributing_factor


In [ ]:
query ="""
MATCH (factor:ContributingFactor {id: "Awareness"})<-[:contributing_factor]-(accident:UniqueAccident)
OPTIONAL MATCH (accident)-[:has_date]->(date:Date)
OPTIONAL MATCH (accident)-[:has_time]->(time:Time)
OPTIONAL MATCH (accident)-[:occurred_in]->(country:Country)
OPTIONAL MATCH (accident)-[:occurred_at]->(section:TrackSection)
OPTIONAL MATCH (accident)-[:is_type]->(type:AccidentType)
OPTIONAL MATCH (accident)-[:investigated_by]->(regulator:RegulatoryBody)

RETURN 
  accident.id AS accident_id,
  date.id AS date,
  time.id AS time,
  country.id AS country,
  section.id AS track_section,
  type.id AS accident_type,
  regulator.id AS regulatory_body
ORDER BY date DESC
"""

run_query(query)

,accident_id,date,time,country,track_section,accident_type,regulatory_body
0,Skierniewice Accident 24/08/2023,24/08/23,12:05,Poland,"railway line no. 1, km 64.282",Collisions,State Commission on Railway Accident Investiga...
1,Chałupki - Krzyżanowice Level Crossing Acciden...,20/02/2023,18:11,Poland,Railway line no. 151 Kędzierzyn-Koźle - Chałup...,Level Crossing Accident involving a train and ...,STATE COMMISSION ON RAILWAY ACCIDENT INVESTIGA...
2,Komorów – Podkowa Leśna Główna Accident on 19/...,19/05/23,09:51,Poland,km 18.805 of railway line no. 47 Warszawa Śród...,Level Crossing Accident,STATE COMMISSION ON RAILWAY ACCIDENT INVESTIGA...


In [ ]:
"""MATCH path = (factor:ContributingFactor {id: "Awareness"})<-[:contributing_factor]-(accident:UniqueAccident)-[r]->(n)
RETURN path"""